In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import nqDataLoader as nq #data loading library
from keras.preprocessing import sequence
import tensorflow as tf
tf.random.set_seed(0)
np.random.seed(0)


ImportError: cannot import name 'to_categorical' from 'keras.utils' (C:\Users\micoa\AppData\Roaming\Python\Python310\site-packages\keras\utils\__init__.py)

In [ ]:
## importing the early stage dataset 
early_stage = pd.read_csv('GT_DataPD_MIT-CS2PD.csv')
# X = dataset.iloc[:, :-1].values
# y = dataset.iloc[:, -1].values
early_stage["gt"] = early_stage["gt"].astype(int)
early_stage.head()

In [ ]:
hit_time_series = []
for index, row in early_stage.iterrows():
    fileloc = row.file_1
    keyPressed, htArr, pressArr, releaseArr =  nq.getDataFiltHelper( "data_MIT-CS2PD/" + early_stage.loc[index]['file_1'])
    htArr =np.array(htArr)
    hit_time_series.append(htArr)

X1 = hit_time_series 

In [ ]:
## importing the de-novo dataset 
de_novo = pd.read_csv('GT_DataPD_MIT-CS1PD.csv')
# X = dataset.iloc[:, :-1].values
# y = dataset.iloc[:, -1].values
print(len(de_novo))
de_novo["gt"] = de_novo["gt"].astype(int)
de_novo.head()

In [ ]:
##using both files 
hit_time_series = []
for index, row in de_novo.iterrows():
    fileloc1 = row.file_1
    keyPressed, htArr1, pressArr, releaseArr =  nq.getDataFiltHelper( 'data_MIT-CS1PD/' + de_novo.loc[index]['file_1'])
    htArr1 = np.array(htArr1)
    keyPressed, htArr2, pressArr, releaseArr =  nq.getDataFiltHelper( 'data_MIT-CS1PD/' + de_novo.loc[index]['file_2'])
    htArr2 = np.array(htArr2)
    htArr =np.concatenate((htArr1,htArr2),axis =0)
    htArr=np.array(htArr)
    hit_time_series.append(htArr)
X2 = hit_time_series 

In [ ]:
X = np.concatenate((X1,X2),axis=0)
y=  np.concatenate((early_stage['gt'],de_novo["gt"]),axis=0)
X_filtered =[]
y_filtered =[]


for i,e in enumerate(X):
    if len(e)>1:
        X_filtered.append(e)
        y_filtered.append(y[i])
min = len(X_filtered[0])
for i,e in enumerate(X_filtered):
    if len(e)<min:
        min = len(e)
        print(i)
X_filtered = np.array(X_filtered)
y_filtered = np.array(y_filtered)
X_lens = [len(e) for e in X_filtered]
avg_len = np.mean(X_lens)
print(avg_len)
print(min)
print(len(X_filtered),len(y_filtered))
print(X_filtered)
print(y_filtered)

In [ ]:
import itertools
flat_list = list(itertools.chain(*X_filtered))
value = sum(flat_list) / len(flat_list)
print(value)
X_filtered =sequence.pad_sequences(X_filtered,dtype='float32',padding='post',maxlen=6000,value =value)



In [ ]:
from pandas import DataFrame
from pandas import concat
 
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    """
    Frame a time series as a supervised learning dataset.
    Arguments:
        data: Sequence of observations as a list or NumPy array.
        n_in: Number of lag observations as input (X).
        n_out: Number of observations as output (y).
        dropnan: Boolean whether or not to drop rows with NaN values.
    Returns:
        Pandas DataFrame of series framed for supervised learning.
    """
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X_filtered,y_filtered,test_size=0.17,random_state=0)

In [ ]:
X_train.shape


In [ ]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range=(0,1))
X_train = sc.fit_transform(X_train)
X_train = np.reshape(X_train, (X_train.shape[0], int(X_train.shape[1]/100),100))
print(X_train)
X_test = sc.transform(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], int(X_test.shape[1]/100),100))

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.callbacks import EarlyStopping
from matplotlib import pyplot
from keras import optimizers


clf = Sequential()
clf.add(LSTM(units = 30,return_sequences = True, input_shape = (X_train.shape[1], X_train.shape[2])))
clf.add(Dropout(0.2))
clf.add(LSTM(units = 30,return_sequences = True))
clf.add(Dropout(0.2))
clf.add(LSTM(units = 30,return_sequences = True))
clf.add(Dropout(0.2))
clf.add(LSTM(units =30 ))
clf.add(Dropout(0.2))
clf.add(Dense(units = 1,activation='sigmoid'))
es = EarlyStopping(monitor='loss', mode='auto', patience =50 ,verbose=1,restore_best_weights=True)
adm = tf.keras.optimizers.Adam(learning_rate=0.0001)
clf.compile(loss='binary_crossentropy', optimizer=adm, metrics=['accuracy'])
print(clf.summary())



history=clf.fit(X_train, y_train, epochs=250, batch_size=35)
scores = clf.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))
train_acc = clf.evaluate(X_train, y_train, verbose=0)
test_acc = clf.evaluate(X_test, y_test, verbose=0)
print('Train: %.3f, Test: %.3f' % (train_acc[1], test_acc[1]))
# plot training history
pyplot.plot(history.history['loss'], label='train loss')
pyplot.plot(history.history['accuracy'], label='train accuracy')
# pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from scikeras.wrappers import KerasClassifier
def LSTM_Network(neurons=30,dropout=0.2):

    clf = Sequential()
    clf.add(LSTM(units = neurons ,return_sequences = True, input_shape = (X_train.shape[1], X_train.shape[2])))
    clf.add(Dropout(dropout))
    clf.add(LSTM(units = neurons ,return_sequences = True))
    clf.add(Dropout(dropout))
    clf.add(LSTM(units = neurons ,return_sequences = True))
    clf.add(Dropout(dropout))
    clf.add(LSTM(units =neurons))
    clf.add(Dropout(dropout))
    clf.add(Dense(units = 1,activation='sigmoid'))
    es = EarlyStopping(monitor='loss', mode='auto', patience =50 ,verbose=1,restore_best_weights=True)
    adm = tf.keras.optimizers.Adam(learning_rate=0.0001)
    clf.compile(loss='binary_crossentropy', optimizer=adm, metrics=['accuracy'])
    
    print(clf.summary())
    return clf
lstm_clf = KerasClassifier(model=LSTM_Network, epochs=300, batch_size=35, verbose=0)
def evaluate_model(model, X, y):
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
    scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
    return scores

In [ ]:
# scores1=evaluate_model(lstm_clf,X_filtered,y_filtered)
# results = [scores1]

In [ ]:
# pyplot.boxplot(results, labels=['RNN'], showmeans=True)
# pyplot.show()

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
import numpy as np
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)
cvscores = []
for train, test in kfold.split(X_filtered, y_filtered):
  # create model
    X_traincv =X_filtered[train]
    y_traincv  = y_filtered[train]
    X_testcv = X_filtered[test]
    y_testcv = y_filtered[test]
    X_traincv = np.log10(X_traincv)
    X_traincv = sc.fit_transform(X_traincv)
    X_traincv = np.reshape(X_traincv, (X_traincv.shape[0], int(X_traincv.shape[1]/60),60))
    X_testcv = np.log10(X_testcv)
    X_testcv = sc.transform(X_testcv)
    X_testcv = np.reshape(X_testcv, (X_testcv.shape[0], int(X_testcv.shape[1]/60),60))
    clf = Sequential()
    clf.add(Conv1D(filters=15, kernel_size=2, padding='same', activation='relu'))
    clf.add(MaxPooling1D(pool_size=2))
    clf.add(Conv1D(filters=15, kernel_size=2, padding='same', activation='relu'))
    clf.add(MaxPooling1D(pool_size=2))
    clf.add(Conv1D(filters=15, kernel_size=2, padding='same', activation='relu'))
    clf.add(MaxPooling1D(pool_size=2))
    clf.add(LSTM(units = 25,return_sequences = True))
    clf.add(Dropout(0.2))
    clf.add(LSTM(units = 25,return_sequences = True))
    clf.add(Dropout(0.2))
    clf.add(LSTM(units = 25,return_sequences = True))
    clf.add(Dropout(0.2))
    clf.add(LSTM(units =25))
    clf.add(Dropout(0.2))
    clf.add(Dense(units = 1,activation='sigmoid'))
    es = EarlyStopping(monitor='loss', mode='auto', patience =50 ,verbose=1,restore_best_weights=True)
    adm = tf.keras.optimizers.Adam(learning_rate=0.0001)
    ada = tf.keras.optimizers.Adadelta(
    learning_rate=0.00001, rho=0.95, epsilon=1e-07)
    bce = tf.keras.losses.BinaryCrossentropy(reduction='sum_over_batch_size')
    clf.compile(loss=bce, optimizer=adm, metrics=['accuracy']) 
    clf.fit(X_traincv, y_traincv, epochs=300, batch_size=35)
    # evaluate the model
    scores = clf.evaluate(X_testcv, y_testcv, verbose=0)
    print("%s: %.2f%%" % (clf.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))
pyplot.boxplot([cvscores], labels=['RNN'], showmeans=True)
pyplot.show()